In [ ]:
#conda env export > environment.yml

replace contractions/location abbreviations via dictionary
denoise text/apply word segmentation
create word count dictionary
next, spell check:
create dictionary from reoccuring words in text? or use freq dict symspell
apply spell check to words in word count dictionary occuring only one time
apply pos tag to tweets creating new column
lemmatize tweets based on pos tag column


In [1]:
import os
import re
import string
import pandas as pd 
import itertools

## import datasets

In [2]:
# import dataset
df = pd.read_csv('train.csv')
#test_df = pd.read_csv('test.csv')
#target = pd.read_csv('train.csv', usecols=['target'])

In [5]:
print(len(df))
#print(len(test_df))

7613


In [7]:
df

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...
7608,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
#test_df.drop(['id'], axis=1, inplace=True)
#df.drop(['id'], axis=1, inplace=True)

In [9]:
#dups = df[df.duplicated(['text'])]
df.drop_duplicates(subset=['text', 'location', 'keyword'], keep='first', inplace=True)
#test_df.drop_duplicates(subset=['text'], keep='first', inplace=True)

print(len(df))
#print(len(test_df))

7552


# process text

## dictionary based word replacement 

In [10]:
import csv

In [ ]:
# import and read previously created dictionary as csv for abbreviations and expansions of words
def csv2dict(csv_name):
    with open(csv_name, mode='r') as f:
        reader = csv.reader(f)
        next(reader)
        return {rows[1]:rows[2] for rows in reader}
    
# import and assign precomposed dictionaries for word replacement for contractions and location initialisms and abbreviations
location_expansion_dict = csv2dict('utils\location_expansion.csv')
contractions_dict = csv2dict('utils\contractions.csv')

## denoise text functions

In [29]:
from symspellpy.symspellpy import SymSpell, Verbosity
from spellchecker import SpellChecker
import pkg_resources
import html

In [53]:
# functions related to expanded hashtags in text
def pascal_case_split(text):
    # expand hashtags formatted in pascal case, ex: #ThisIsPascal
    return re.sub(r'([A-Z])([?=a-z0-9+])', r' \1\2', text)

def camel_case_split(text):
    # expand hashtags formatted in pascal case, ex: #thisIsCamel
    return re.sub(r'([a-z0-9+])([?<=A-Z])', r'\1 \2', text)

def replace_spell(text):
    r_re = re.compile(r'\b'+r'\b|\b'.join(map(re.escape, filtered_repl.keys()))+r'\b')
    def replace(match):
        replacement =  f"{filtered_repl[match.group(0)]}"
        return replacement
    text = r_re.sub(replace, text)
    return text

def spell_check_word(word):
    # lookup suggestions for individual words
    spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename(
                      "symspellpy", "frequency_dictionary_en_82_765.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    corrections = spell.lookup(word, Verbosity.TOP, include_unknown=True)
    closest = corrections[0]
    # display suggestion term, edit distance, and term frequency
    return closest.term
        
def spell_check_words(word):
    # lookup suggestions for multi word string input
    spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=8)
    dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
    bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
    corrections = spell.lookup_compound(word, max_edit_distance=2)
    closest = corrections[0]
# display suggestion term, edit distance, and term frequency
    return closest.term

def desegment_strings(text):
    spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=8)
    dictionary_path = pkg_resources.resource_filename("symspellpy","frequency_dictionary_en_82_765.txt")
    spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    if text.isupper() | text.islower() and text.startswith('#'):
        result = spell.word_segmentation(text)
        text = result.corrected_string
    return text 

def remove_hash(text):
    # remove hash symbol in front of hashtag text and remove non unicode chars
    return re.sub('#', '', text)

def expand_strings(text):
    # combine hashtag split functions for specific cases and remove hash
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    h_text = remove_hash(p_text)
    return h_text

def expand_strings_sc(text):
    # combine hashtag split functions for specific cases and remove hash, spell check using compound lookup
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    lu_text = spell_check_words(p_text)
    h_text = remove_hash(lu_text)
    return h_text

def expand_strings_ds(text):
    # combine hashtag split functions for specific cases and remove hash. spell check using wordsegmentation
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    lu_text = desegment_strings(p_text)
    h_text = remove_hash(lu_text)
    return h_text

def expand_strings_indv_word(text):
    # combine hashtag split functions for specific cases and remove hash. spell check using wordsegmentation
    c_text = camel_case_split(text)
    p_text = pascal_case_split(c_text)
    lu_text = spell_check_word(p_text)
    h_text = remove_hash(lu_text)
    return h_text


def word_expansion_from_dict(text, expansion_dict):
    c_re = re.compile(r'\b'+r'\b|\b'.join(map(re.escape, expansion_dict.keys()))+r'\b')
    def replace(match):
        expansion =  f"{expansion_dict[match.group(0)]}"
        return expansion
    text = c_re.sub(replace, text)
    return text

def reduce_repeat_vowels(text, keep_reps=1):
    # function to reduce repeated vowel occurences in string, 
    # keep_reps argument as int refers to number of occurences to keep in string
    # keep_reps argument default : 1, can be set to any integer
    vowels = ['a','e','i','o','u']
    for ch in iter(vowels):
        text = re.sub(r'(?i)'+rf'([{ch}])'+r'{3,}', r'\1'*int(keep_reps), text)
    return text

def reduce_repeat_consonants(text, keep_reps=2):
    # function to reduce repeated consonant occurences in string, 
    # keep_reps argument as int refers to number of occurences to keep in string
    # keep_reps argument default : 1, can be set to any integer
    consonants = ['b','c','d','f','g','h','j','k','l','m','n','p','q','r','s','t','v','w','x','y','z']
    for ch in iter(consonants):
        text = re.sub(r'(?i)'+ rf'([{ch}])'+r'{3,}', r'\1'*int(keep_reps), text)
    return text
    
def remove_urls(text):
    # function to remove urls from string
    return re.sub(r"http\S+",  r"", text)

def remove_handles(text):
    # function to remove twitter handles from string
    return re.sub("@\S*",  r" ", text)

def remove_non_uni(text):
    # function to remove non unicode characters from string
    return re.sub('[^\u0000-\u007f]', ' ', text)

def remove_nan_str(text):
    # function to remove 'nan' from string
    return re.sub('nan', '', text)

def remove_ws(text):
    return re.sub(' +', ' ', text)

def remove_non_alpha(text):
    # function to remove non alpha chars from string
    return re.sub("[^a-zA-Z]",  r" ", text)

def denoise_text(text):
    #reduce repeating characters
    text = reduce_repeat_vowels(text)
    text = reduce_repeat_consonants(text)
    #remove xml tag strings
    text = html.unescape(text)
    #remove numeric and punctuation chars
    text = remove_non_alpha(text)
    #remove extra spaces
    text = remove_ws(text)
    return text   
    

## tweet text denoise 

In [31]:
#### text preprocessing specifically formatted for tweets but will work on any text
def process_tweet(df): 
    """combine regex and nltk processing for tweet text and keyword text processing"""
    def expand_strings_ds(text):
        # combine hashtag split functions for specific cases and remove hash
        # remove urls and usernames first due to alternating casing inside before case format expansion
        u_text = remove_urls(text)
        hd_text = remove_handles(u_text)
        c_text = camel_case_split(hd_text)
        p_text = pascal_case_split(c_text)
        lu_text = desegment_strings(p_text)
        h_text = remove_hash(lu_text)
        return h_text
    
    def expand_text(text):
        e_text = expand_strings_ds(text)
        le_text = word_expansion_from_dict(e_text, expansion_dict=location_expansion_dict)
        ce_text = word_expansion_from_dict(le_text.lower(), expansion_dict=contractions_dict)
        return ce_text

    def process_text(text):
        exp_text = expand_text(text)
        clean_text = denoise_text(exp_text)
        return clean_text

    processed_df = [process_text(x) for x in df]
    return processed_df

## keyword text denoise process

In [32]:
def process_keyword(df):
    text = df.str.replace('%20', ' ')
    return text

## location text denoise process

In [33]:
def process_location(df):
    """combine regex and nltk processing for location text processing"""
    
    def expand_loc_text(text):
        #exp_text = expand_strings_ds(text) 
        loc_dict_lower = {k.lower(): v for k, v in location_expansion_dict.items()}
        exp_text = word_expansion_from_dict(text.lower(), loc_dict_lower)
        return exp_text 
        
    def process_text(text):
        exp_text = expand_loc_text(text)
        clean_text = denoise_text(exp_text)
        return clean_text
    
    loc_df = [process_text(x) for x in df]
    return loc_df 

# create new dataframe of denoised text

In [35]:
def scrub_data_to_df(df):
    df['tweets'] = process_tweet(df.text)
    df['clean_keyword'] = process_keyword(df.keyword.astype(str))
    df['clean_location'] = process_location(df.location.astype(str))
    return df

## use frequency dictionary for text to spell correct

In [37]:
from collections import Counter
from nltk.probability import FreqDist
from operator import itemgetter

In [ ]:
def spell_check_rare(df):
    corpus = df.tweets.str.split()
    word_dict = Counter(corpus.sum())
    single_occ_words = [k for k,v in word_dict.items() if int(v) <= 1]
    corrections = []
    for x in single_occ_words:
        repl = spell_check_words(x)
        corrections.append(repl)
    replacement_dict = dict(zip(single_occ_words, corrections))
    filtered_repl = dict(filter(lambda elem: elem[0] != elem[1], replacement_dict.items()))
    def replace_spell(text):
        r_re = re.compile(r'\b'+r'\b|\b'.join(map(re.escape, filtered_repl.keys()))+r'\b')
        def replace(match):
            replacement =  f"{filtered_repl[match.group(0)]}"
            return replacement
        text = r_re.sub(replace, text)
        return text
    df['new'] = df['tweets'].apply(replace_spell)
    return df

In [56]:
def spell_check_compound(df):
    corpus = df.tweets.str.split()
    word_dict = Counter(corpus.sum())
    single_occ_words = [k for k,v in word_dict.items() if int(v) <= 1]
    corrections = []
    for x in single_occ_words:
        repl = spell_check_words(x)
        corrections.append(repl)
    replacement_dict = dict(zip(single_occ_words, corrections))
    filtered_repl = dict(filter(lambda elem: elem[0] != elem[1], replacement_dict.items()))
    def replace_spell(text):
        r_re = re.compile(r'\b'+r'\b|\b'.join(map(re.escape, filtered_repl.keys()))+r'\b')
        def replace(match):
            replacement =  f"{filtered_repl[match.group(0)]}"
            return replacement
        text = r_re.sub(replace, text)
        return text
    df['new'] = df['tweets'].apply(replace_spell)
    return df

In [128]:
df.tweets

0        our deeds are the reason of this earthquake m...
1                   forest fire near la ronge sask canada
2        all residents asked to shelter in place are b...
3        people receive wildfires evacuation orders in...
4        just got sent this photo from ruby alaska as ...
                              ...                        
7547     two giant cranes holding a bridge collapse in...
7548     the out of control wild fires in california e...
7549                        m utc km s of volcano hawaii 
7550     police investigating after an e bike collided...
7551     the latest more homes razed by northern calif...
Name: tweets, Length: 7552, dtype: object

In [175]:
def get_corpus(df):
    tweets = df['tweets']
    corpus = ' '.join(word for word in tweets)
    return corpus

In [176]:
def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

corpus = get_corpus(train_df2)

word_count_dict = word_count(corpus)


In [177]:

len(word_count_dict)

13685

In [191]:
df

,Unnamed: 0,target,tweets,clean_keyword,clean_location,new,clean_tweet,all_text,combined_tokens,tweet_tokens,filtered,tokens
0,0,1,our deeds are the reason of this earthquake m...,,,our deeds are the reason of this earthquake m...,deed reason earthquake may allah forgive,deed reason earthquake may allah forgive,"['deed', 'reason', 'earthquake', 'may', 'allah...","['deed', 'reason', 'earthquake', 'may', 'allah...","['deed', 'reason', 'earthquake', 'may', 'allah...","[our, deeds, are, the, reason, of, this, earth..."
1,1,1,forest fire near la ronge sask canada,,,forest fire near la range sask canada,forest fire near range sask canada,forest fire near range sask canada,"['forest', 'fire', 'near', 'range', 'sask', 'c...","['forest', 'fire', 'near', 'range', 'sask', 'c...","['forest', 'fire', 'near', 'range', 'canada']","[forest, fire, near, la, ronge, sask, canada]"
2,2,1,all residents asked to shelter in place are b...,,,all residents asked to shelter in place are b...,resident ask shelter place notify officer evac...,resident ask shelter place notify officer evac...,"['resident', 'ask', 'shelter', 'place', 'notif...","['resident', 'ask', 'shelter', 'place', 'notif...","['resident', 'ask', 'shelter', 'place', 'offic...","[all, residents, asked, to, shelter, in, place..."
3,3,1,people receive wildfires evacuation orders in...,,,people receive wildfires evacuation orders in...,people receive wildfire evacuation order calif...,people receive wildfire evacuation order calif...,"['people', 'receive', 'wildfire', 'evacuation'...","['people', 'receive', 'wildfire', 'evacuation'...","['people', 'receive', 'wildfire', 'evacuation'...","[people, receive, wildfires, evacuation, order..."
4,4,1,just got sent this photo from ruby alaska as ...,,,just got sent this photo from ruby alaska as ...,get send photo ruby alaska smoke wildfire pour...,get send photo ruby alaska smoke wildfire pour...,"['get', 'send', 'photo', 'ruby', 'alaska', 'sm...","['get', 'send', 'photo', 'ruby', 'alaska', 'sm...","['get', 'send', 'photo', 'ruby', 'alaska', 'sm...","[just, got, sent, this, photo, from, ruby, ala..."
...,...,...,...,...,...,...,...,...,...,...,...,...
7547,7608,1,two giant cranes holding a bridge collapse in...,,,two giant cranes holding a bridge collapse in...,two giant crane hold bridge collapse nearby home,two giant crane hold bridge collapse nearby home,"['two', 'giant', 'crane', 'hold', 'bridge', 'c...","['two', 'giant', 'crane', 'hold', 'bridge', 'c...","['two', 'giant', 'crane', 'hold', 'bridge', 'c...","[two, giant, cranes, holding, a, bridge, colla..."
7548,7609,1,the out of control wild fires in california e...,,,the out of control wild fires in california e...,control wild fire california even northern par...,control wild fire california even northern par...,"['control', 'wild', 'fire', 'california', 'eve...","['control', 'wild', 'fire', 'california', 'eve...","['control', 'wild', 'fire', 'california', 'eve...","[the, out, of, control, wild, fires, in, calif..."
7549,7610,1,m utc km s of volcano hawaii,,,m utc km s of volcano hawaii,utc volcano hawaii,utc volcano hawaii,"['utc', 'volcano', 'hawaii']","['utc', 'volcano', 'hawaii']","['utc', 'volcano', 'hawaii']","[m, utc, km, s, of, volcano, hawaii]"
7550,7611,1,police investigating after an e bike collided...,,,police investigating after an e bike collided...,police investigate bike collide car little por...,police investigate bike collide car little por...,"['police', 'investigate', 'bike', 'collide', '...","['police', 'investigate', 'bike', 'collide', '...","['police', 'investigate', 'bike', 'collide', '...","[police, investigating, after, an, e, bike, co..."


In [194]:

# count number of words in corpus
num_words = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words} words in the entire training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
print(f'There are {len(word_count_sorted)} unique words in the training dataset')

single_words_list= [k for k,v in word_dict.items() if int(v) <=2]

There are 106022 words in the entire training dataset
There are 13683 unique words in the training dataset


In [195]:
single_words_list

['deeds',
 'forgive',
 'ronge',
 'sask',
 'notified',
 'receive',
 'ruby',
 'pours',
 'directions',
 'afire',
 'manitou',
 'woods',
 'fvck',
 'count',
 'bago',
 'fruits',
 'skiing',
 'pasta',
 'africanbaze',
 'mufc',
 'acquisitions',
 'epl',
 'inec',
 'abia',
 'barbados',
 'bridgetown',
 'elizabeth',
 'superintende',
 'nsfw',
 'visiting',
 'cfc',
 'ancop',
 'tita',
 'vida',
 'pumped',
 'preaching',
 'tracklist',
 'retainers',
 'brighton',
 'superintendent',
 'lanford',
 'deliberately',
 'noches',
 'bestia',
 'teammates',
 'goodnight',
 'gunners',
 'kurds',
 'trampling',
 'turkmen',
 'vandalized',
 'offices',
 'diyala',
 'voortrekker',
 'tambo',
 'intl',
 'skyline',
 'kiss',
 'lips',
 'revel',
 'wmv',
 'farewell',
 'gtx',
 'wm',
 'progressive',
 'greetings',
 'pens',
 'publications',
 'rene',
 'jacinta',
 'edit',
 'mar',
 'tinderbox',
 'hood',
 'buff',
 'magnitude',
 'testicles',
 'clara',
 'pkwy',
 'personalinjury',
 'advice',
 'solicitor',
 'otley',
 'stlouis',
 'caraccidentlawyer',
 

In [186]:

len(single_words_list )


7385

## create column containing combined column text & tokens columns

In [159]:
def combine_columns(df):
    df['clean_keyword'] = df.clean_keyword.astype(str)
    df['clean_keyword'] = df.clean_keyword.str.lower()
    df['clean_keyword'] = df.clean_keyword.str.replace(r'\bnan\b', r'')
    df['clean_keyword'] = df.clean_keyword.str.replace(r' ', r'_')
    df['clean_keyword'] = df.clean_keyword.str.strip()
    df['clean_location'] = df.clean_location.astype(str)
    df['clean_location'] = df.clean_location.str.lower()
    df['clean_location'] = df.clean_location.str.strip()
    df['clean_location'] = df.clean_location.str.replace(r'\bnan\b', r'')
    df['clean_location'] = df.clean_location.str.replace(r' ', r'_')
    #df['tweets'] = df.tweets.astype(str)
    df['tokens'] = df.tweets.apply(nltk.word_tokenize)
    #df['combined_tokens'] = df.combined_tokens.apply(lambda row: row.split())
    # 
    new_df = df
    return new_df

## check single occurence words again/remove words used only once

In [42]:
def remove_single_use_words(tokens):
    # function to remove stopwords from text
    stop_list = filtered_remove 
    new_tokens = []
    for word in tokens:
        if word not in stop_list:
            new_tokens.append(word)
    return new_tokens

In [125]:
tweets = train_df.tokens
word_dict = Counter(tweets.sum())
# count number of words in corpus
num_words = sum(word_dict[w] for w in word_dict)
print(f'There are {num_words} words in the entire training dataset')

# count number of unique words in corpus
word_count_sorted = [(value, key) for key, value in word_dict.items()]
word_count_sorted.sort()
print(f'There are {len(word_count_sorted)} unique words in the training dataset')

There are 106022 words in the entire training dataset
There are 13683 unique words in the training dataset


## process  data

In [43]:
scrub_data_to_df(df)
df.to_csv('train_dn_pt.csv')
spell_check_rare(df)
df.to_csv('train_scr_pt.csv')
#test_df['clean_tweet'] = test_df['new'].apply(tokenize_tweet)

In [57]:
scc = spell_check_compound(df)
scc.to_csv('train_sc_compound.csv')

In [152]:
scc = pd.read_csv('train_sc_compound.csv')
df = pd.read_csv('train_tokenized.csv')

In [157]:
import numpy as np
idx = np.where((train_df['new']!=train_df['tweets']) )
idx

(array([   1,   11,   33, ..., 7530, 7534, 7541], dtype=int64),)

In [158]:
diff = train_df.loc[idx]
len(diff)

1916

In [160]:
train_df2 = combine_columns(scc)
train_tweets2 = scc.tokens
train_word_dict2 = Counter(train_tweets2.sum())
filtered_mono2 = dict(filter(lambda elem: elem[1] == 1, train_word_dict2.items()))

<ipython-input-159-4ab27c84be16>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_keyword'] = df.clean_keyword.str.replace(r'\bnan\b', r'')
<ipython-input-159-4ab27c84be16>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_location'] = df.clean_location.str.replace(r'\bnan\b', r'')


In [161]:
train_df = combine_columns(df)
train_tweets = train_df.tokens
train_word_dict = Counter(train_tweets.sum())
filtered_mono = dict(filter(lambda elem: elem[1] == 1, train_word_dict.items()))

<ipython-input-159-4ab27c84be16>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_keyword'] = df.clean_keyword.str.replace(r'\bnan\b', r'')
<ipython-input-159-4ab27c84be16>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_location'] = df.clean_location.str.replace(r'\bnan\b', r'')


In [162]:
filtered_mono == filtered_mono2

True

In [163]:
scrub_data_to_df(test_df)
test_df.to_csv('test_dn_pt.csv')
spell_check_rare(test_df)
test_df.to_csv('test_scr_pt.csv')

In [170]:
test_df = combine_columns(test_df)

<ipython-input-169-1a87af0fd212>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_keyword'] = df.clean_keyword.str.replace(r'\bnan\b', r'')
<ipython-input-169-1a87af0fd212>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_location'] = df.clean_location.str.replace(r'\bnan\b', r'')


In [184]:
test_tweets = test_df.tweet_tokens
test_word_dict = Counter(test_tweets.sum())
filtered_remove = dict(filter(lambda elem: elem[1] == 1, test_word_dict.items()))

In [189]:
test_df['filtered'] = test_df.tweet_tokens.apply(remove_single_use_words)

In [193]:
test_df

,id,hashtags,tweets,clean_keyword,clean_location,new,clean_tweet,all_text,combined_tokens,tweet_tokens,filtered
0,0,[],just happened a terrible car crash,,,just happened a terrible car crash,happen terrible car crash,happen terrible car crash,"[happen, terrible, car, crash]","[happen, terrible, car, crash]","[happen, terrible, car, crash]"
1,2,['#earthquake'],heard about earthquake is different cities st...,,,heard about earthquake is different cities st...,heard earthquake different city stay safe ever...,heard earthquake different city stay safe ever...,"[heard, earthquake, different, city, stay, saf...","[heard, earthquake, different, city, stay, saf...","[heard, earthquake, different, city, stay, saf..."
2,3,[],there is a forest fire at spot pond geese are ...,,,there is a forest fire at spot pond geese are ...,forest fire spot pond goose flee across street...,forest fire spot pond goose flee across street...,"[forest, fire, spot, pond, goose, flee, across...","[forest, fire, spot, pond, goose, flee, across...","[forest, fire, spot, pond, across, street, save]"
3,9,"['#Spokane', '#wildfires']",apocalypse lighting spokane wildfires,,,apocalypse lighting spokane wildfires,apocalypse light spokane wildfire,apocalypse light spokane wildfire,"[apocalypse, light, spokane, wildfire]","[apocalypse, light, spokane, wildfire]","[apocalypse, light, wildfire]"
4,11,[],typhoon soudelor kills in china and taiwan,,,typhoon soudelor kills in china and taiwan,typhoon soudelor kill china taiwan,typhoon soudelor kill china taiwan,"[typhoon, soudelor, kill, china, taiwan]","[typhoon, soudelor, kill, china, taiwan]","[typhoon, soudelor, kill, china, taiwan]"
...,...,...,...,...,...,...,...,...,...,...,...
3258,10861,[],earthquake safety los angeles safety fasteners...,,,earthquake safety los angeles safety fasteners...,earthquake safety los angeles safety fastener,earthquake safety los angeles safety fastener,"[earthquake, safety, los, angeles, safety, fas...","[earthquake, safety, los, angeles, safety, fas...","[earthquake, safety, los, angeles, safety]"
3259,10865,[],storm in rhode island worse than last hurrica...,,,storm in rhode island worse than last hurrica...,storm rhode island bad last hurricane city oth...,storm rhode island bad last hurricane city oth...,"[storm, rhode, island, bad, last, hurricane, c...","[storm, rhode, island, bad, last, hurricane, c...","[storm, rhode, island, bad, last, hurricane, c..."
3260,10868,[],green line derailment in chicago,,,green line derailment in chicago,green line derailment chicago,green line derailment chicago,"[green, line, derailment, chicago]","[green, line, derailment, chicago]","[green, line, derailment, chicago]"
3261,10874,[],meg issues hazardous weather outlook hwo,,,meg issues hazardous weather outlook hwo,meg issue hazardous weather outlook hwo,meg issue hazardous weather outlook hwo,"[meg, issue, hazardous, weather, outlook, hwo]","[meg, issue, hazardous, weather, outlook, hwo]","[meg, issue, hazardous, weather, outlook, hwo]"
